# CrysFML: CIF-based vs file-less interfaces

In [1]:
# Misc
import sys, os
import numpy as np

# CrysFML
import CFML_api

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [2]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

Loading BokehJS ...

In [3]:
def diffraction_pattern_xy(space_group, cell, atom_list, job_info):
    reflection_list = CFML_api.ReflectionList(cell, space_group, 
                                              True, 
                                              job_info)
    reflection_list.compute_structure_factors(space_group, 
                                              atom_list, 
                                              job_info)
    diffraction_pattern = CFML_api.DiffractionPattern(job_info,
                                                  reflection_list, 
                                                  cell.reciprocal_cell_vol)
    return diffraction_pattern.x, diffraction_pattern.ycalc

In [4]:
def set_default_parameters(job_info):
    job_info.range_2theta = (10.0, 160.0)
    job_info.theta_step = 0.03
    job_info.lambdas = (1.5405, 1.5405)
    job_info.lambda_ratio = 1.0
    job_info.u_resolution = 0.0509
    job_info.v_resolution = -0.08 # -0.0992 from PCR leads to empty regions...
    job_info.w_resolution = 0.0395
    job_info.x_resolution = 0.1491
    job_info.y_resolution = 0.0

## CIF-based interface

### Create CIF object

In [5]:
cif_obj = CFML_api.CIFFile('../datasets/xrays_powder_PbSO4/PbSO4.cif')

### Create space group object

In [6]:
space_group = cif_obj.space_group
space_group.print_description()



        Information on Space Group: 
        --------------------------- 

 =>   Number of Space group:  62
 =>  Hermann-Mauguin Symbol: P n m a
 =>             Hall Symbol: -P 2ac 2n
 =>            Setting Type: IT (Generated from Hermann-Mauguin symbol)
 =>          Crystal System: Orthorhombic
 =>              Laue Class: mmm
 =>             Point Group: mmm
 =>         Bravais Lattice: P
 =>          Lattice Symbol: oP
 =>  Reduced Number of S.O.:   4
 =>    General multiplicity:   8
 =>          Centrosymmetry: Centric (-1 at origin)
 =>  Generators (exc. -1&L):   2
 =>         Asymmetric unit:  0.000 <= x <=  0.500
                              0.000 <= y <=  0.250
                              0.000 <= z <=  1.000
 => Centring vectors:  0
                                                                                                    

 => List of all Symmetry Operators and Symmetry Symbols

 => SYMM(  1): x,y,z                             Symbol: 1
 => SYMM(  2): x+1/2,-y+

### Create unit cell object

In [7]:
unit_cell = cif_obj.cell
unit_cell.print_description()


        Metric information:
        -------------------

 => Direct cell parameters:

         a =       8.4770      b =       5.3960      c =       6.9570
     alpha =       90.000   beta =       90.000  gamma =       90.000
                        Direct Cell Volume =     318.2263

 => Reciprocal cell parameters:

         a*=     0.117966      b*=     0.185322      c*=     0.143740
     alpha*=       90.000   beta*=       90.000  gamma*=       90.000
                    Reciprocal Cell Volume =   0.00314242

 => Direct and Reciprocal Metric Tensors:

                   GD                                       GR
     71.8595      0.0000      0.0000          0.013916    0.000000    0.000000
      0.0000     29.1168      0.0000          0.000000    0.034344    0.000000
      0.0000      0.0000     48.3999          0.000000    0.000000    0.020661

 =>  Cartesian frame: z // c; y is in the bc-plane; x is along y ^ z = a*  

     Crystal_to_Orthonormal_Matrix              Orthonormal_t

### Create list of atoms

In [8]:
atom_list = cif_obj.atom_list
atom_list.print_description()


        Atoms information:
        ------------------

    Atom      Chem        x/a       y/b       z/c       Biso     Occ       Mult
    Pb         Pb       0.1880    0.2500    0.1668    1.3344    0.5000        4  None
    S          S        0.0650    0.2500    0.6850    0.1974    0.5000        4  None
    O1         O        0.9080    0.2500    0.5950    1.9423    0.5000        4  None
    O2         O        0.1935    0.2500    0.5432    1.4212    0.5000        4  None
    O3         O        0.0814    0.0274    0.8085    1.3028    1.0000        8  None


### Create job definition object

In [9]:
job_info = cif_obj.job_info
job_info.print_description()

 General Job: CrysFML
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases:  General Job: CrysFML                                                                                                           
Name of phases:  General Job: CrysFML                                                                                                           
Lambda range: (1.5405000448226929, 1.5405000448226929)
Lambda ratio: 0.0
Range 2theta: (0.0, 120.0)
Range sin(theta)/lambda: (0.0, 0.5621716380119324)


### Modify job definition object

In [10]:
#job_info.pattern_type = "NEUT_2THE"
set_default_parameters(job_info)
job_info.print_description()

 General Job: CrysFML
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases:  General Job: CrysFML                                                                                                           
Name of phases:  General Job: CrysFML                                                                                                           
Lambda range: (1.5405000448226929, 1.5405000448226929)
Lambda ratio: 1.0
Range 2theta: (10.0, 160.0)
Range sin(theta)/lambda: (0.05657626688480377, 0.6392779350280762)


### Plot calculated pattern

In [11]:
x_data, y_data_cif_xrays = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_cif_xrays/200, legend_label='Xrays (CIF-based) / 200', color='dodgerblue', line_width=2)
show(fig)

 XRAY_2THE       


## File-less interface

### Create space group object

In [12]:
space_group = CFML_api.SpaceGroup('P n m a')
space_group.print_description()



        Information on Space Group: 
        --------------------------- 

 =>   Number of Space group:  62
 =>  Hermann-Mauguin Symbol: P n m a
 =>             Hall Symbol: -P 2ac 2n
 =>            Setting Type: IT (Generated from Hermann-Mauguin symbol)
 =>          Crystal System: Orthorhombic
 =>              Laue Class: mmm
 =>             Point Group: mmm
 =>         Bravais Lattice: P
 =>          Lattice Symbol: oP
 =>  Reduced Number of S.O.:   4
 =>    General multiplicity:   8
 =>          Centrosymmetry: Centric (-1 at origin)
 =>  Generators (exc. -1&L):   2
 =>         Asymmetric unit:  0.000 <= x <=  0.500
                              0.000 <= y <=  0.250
                              0.000 <= z <=  1.000
 => Centring vectors:  0
                                                                                                    

 => List of all Symmetry Operators and Symmetry Symbols

 => SYMM(  1): x,y,z                             Symbol: 1
 => SYMM(  2): x+1/2,-y+

### Create unit cell object

In [13]:
lengths = np.asarray([8.477,5.396,6.957], dtype='float32')
angles = np.asarray([90,90,90], dtype='float32')
unit_cell = CFML_api.Cell(lengths, angles)
unit_cell.print_description()


        Metric information:
        -------------------

 => Direct cell parameters:

         a =       8.4770      b =       5.3960      c =       6.9570
     alpha =       90.000   beta =       90.000  gamma =       90.000
                        Direct Cell Volume =     318.2263

 => Reciprocal cell parameters:

         a*=     0.117966      b*=     0.185322      c*=     0.143740
     alpha*=       90.000   beta*=       90.000  gamma*=       90.000
                    Reciprocal Cell Volume =   0.00314242

 => Direct and Reciprocal Metric Tensors:

                   GD                                       GR
     71.8595      0.0000      0.0000          0.013916    0.000000    0.000000
      0.0000     29.1168      0.0000          0.000000    0.034344    0.000000
      0.0000      0.0000     48.3998          0.000000    0.000000    0.020661

 =>  Cartesian frame: z // c; y is in the bc-plane; x is along y ^ z = a*  

     Crystal_to_Orthonormal_Matrix              Orthonormal_t

### Create list of atoms

In [14]:
cif_str = '''
loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_occupancy
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb  1.0  0.1880  0.25    0.1668  Uiso  0.0169
  S   S   1.0  0.0650  0.25    0.6850  Uiso  0.0025
  O1  O   1.0  0.9080  0.25    0.5950  Uiso  0.0246
  O2  O   1.0  0.1935  0.25    0.5432  Uiso  0.0180
  O3  O   1.0  0.0814  0.0274  0.8085  Uiso  0.0165
'''
atom_list = CFML_api.AtomList(cif_str.split('\n'))
atom_list.set_mult_occ_cif(space_group)
atom_list.print_description()


        Atoms information:
        ------------------

    Atom      Chem        x/a       y/b       z/c       Biso     Occ       Mult
    Pb         Pb       0.1880    0.2500    0.1668    1.3344    0.5000        4  None
    S          S        0.0650    0.2500    0.6850    0.1974    0.5000        4  None
    O1         O        0.9080    0.2500    0.5950    1.9423    0.5000        4  None
    O2         O        0.1935    0.2500    0.5432    1.4212    0.5000        4  None
    O3         O        0.0814    0.0274    0.8085    1.3028    1.0000        8  None


### Create job definition object

In [15]:
cfl_str = '''
Title SrTiO3
'''
job_info = CFML_api.JobInfo(cfl_str.split('\n'))
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.5405000448226929, 1.5405000448226929)
Lambda ratio: 0.0
Range 2theta: (0.0, 120.0)
Range sin(theta)/lambda: (0.0, 0.5621716380119324)


### Modify job definition object

In [16]:
#job_info.pattern_type = "NEUT_2THE"
set_default_parameters(job_info)
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.5405000448226929, 1.5405000448226929)
Lambda ratio: 1.0
Range 2theta: (10.0, 160.0)
Range sin(theta)/lambda: (0.05657626688480377, 0.6392779350280762)


### Plot calculated pattern

In [17]:
x_data, y_data_code_xrays = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_code_xrays/200, legend_label='Xrays (file-less) / 200', color='olivedrab', line_width=2)
show(fig)

 XRAY_2THE       


### Compare X-rays from CIF and programatically

In [18]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_cif_xrays/200, legend_label='Xrays (CIF-based) / 200', color='dodgerblue', line_width=2)
fig.line(x_data, y_data_code_xrays/200, legend_label='Xrays (file-less) / 200', color='olivedrab', line_width=2)
fig.line(x_data, y_data_cif_xrays/200 - y_data_code_xrays/200, legend_label='Difference', color='orange', line_width=2)
show(fig)